How does the upper atmospheric sensitivity vary across the bootstrapped sample?

In [ ]:
from jacobian import *
from src.data import open_data
import common

In [ ]:
# bootstrap sample size
n = 200

training = open_data('training')
training['region'] = common.get_regions(training.y)
tropics = training.isel(y=slice(30,34)).load()
tropics['time_of_day'] = tropics.time % 1
p = tropics.p[0].values

Generate the bootstrapped samples:

In [ ]:
model = get_model(path="../../models/265/5.pkl")
samples = list(bootstrap_samples(tropics, n))
jacobians = [get_jacobian(model, sample) for sample in samples]

# Compute the heating sensitivity

One metric of the sensitivity is given by

$$\sqrt{\left\langle \left|\frac{\partial Q_1}{\partial q_T(200 mb)}\right|^2 \right\rangle} $$

where $\langle \cdot \rangle$ is a vertically mass-weighted average.

In [ ]:
def heating_sensitivity_to_upper_humidity(jacobian, p):
    """Heating sensitivity in W/m2/(g/kg)
    """
    sensitivity = jacobian['SLI']['QT'].detach().numpy()
    mass = tropics.layer_mass[0].values[:, None]
    return np.sqrt((sensitivity[:, 20]**2 * mass).sum()/mass.sum())

In [ ]:
vars_to_compare = ['PW', 'time_of_day', 'LHF']

data = {'sensitivity': [heating_sensitivity_to_upper_humidity(jac, p) for jac in jacobians]}

for variable in vars_to_compare:
    data[variable] = [float(sample[variable]) for sample in samples]


df = pd.DataFrame(data)

We can see that sensitivity is large accross the bootstrap sample

In [ ]:
df.sensitivity.describe()

This can be seen from looking at the histogram

In [ ]:
df.sensitivity.plot(kind='hist')
plt.xlabel('Sensitivity (K/day/(g/kg))');

In [ ]:
df.plot(kind='scatter', x='PW', y='sensitivity')

The sensitivity clearly grows with PW, just as Prec does.